# 

In [1]:
from sqlalchemy import text
import pandas as pd
from connect import set_connection

In [2]:
with set_connection() as pg:
    pg.execute(text('select 1'))

In [6]:
dates_range = pd.date_range(start = '2021-03-31', end = '2024-12-31', freq='1m')

C:\Users\student\AppData\Local\Temp\ipykernel_10620\1235740707.py:1: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates_range = pd.date_range(start = '2021-03-31', end = '2024-12-31', freq='1m')


In [7]:
dates_range

DatetimeIndex(['2021-03-31', '2021-04-30', '2021-05-31', '2021-06-30',
               '2021-07-31', '2021-08-31', '2021-09-30', '2021-10-31',
               '2021-11-30', '2021-12-31', '2022-01-31', '2022-02-28',
               '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30',
               '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31',
               '2022-11-30', '2022-12-31', '2023-01-31', '2023-02-28',
               '2023-03-31', '2023-04-30', '2023-05-31', '2023-06-30',
               '2023-07-31', '2023-08-31', '2023-09-30', '2023-10-31',
               '2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29',
               '2024-03-31', '2024-04-30', '2024-05-31', '2024-06-30',
               '2024-07-31', '2024-08-31', '2024-09-30', '2024-10-31',
               '2024-11-30', '2024-12-31'],
              dtype='datetime64[ns]', freq='ME')

In [9]:
def query(ix, month_end):
	query = f"""
	{'create table rfm as' if ix == 0 else 'insert into rfm'}
	with
	base as (
		select 
			date'{month_end}' as rep_date
			, customer_id
			, to_char(date'{month_end}', 'yyyymm')::integer as monthkey
			, max(invoice_date) as last_purchase_date
			, extract('day' from date'{month_end}' - max(invoice_date)) as days_since
			, count(distinct invoice_date) as activity_cnt
			, sum(total) as total_sum
		from invoice
		where
			invoice_date between
				date_trunc('month', date'{month_end}' - make_interval(months=>2))
				and date'{month_end}'
		group by
			customer_id
	)
	, rankings as (
		select 
			base.*
			, ntile(4) over(order by days_since desc) as recency
			, ntile(4) over(order by activity_cnt) as frequency
			, ntile(4) over(order by total_sum) as monetary_value
		from base
	)
	select
		r.*
		, recency * 100 + frequency * 10 + monetary_value as rfm_score
	from rankings r
	"""

	return query

In [ ]:
for ix, month_end in enumerate(dates_range):
    print(f"executing for day {month_end}...")
    with set_connection() as pg:
        pg.execute(text(
            query(ix, month_end)
        ))
        pg.commit()

executing for day 2021-03-31 00:00:00...
executing for day 2021-04-30 00:00:00...
executing for day 2021-05-31 00:00:00...
executing for day 2021-06-30 00:00:00...
executing for day 2021-07-31 00:00:00...
executing for day 2021-08-31 00:00:00...
executing for day 2021-09-30 00:00:00...
executing for day 2021-10-31 00:00:00...
executing for day 2021-11-30 00:00:00...
executing for day 2021-12-31 00:00:00...
executing for day 2022-01-31 00:00:00...
executing for day 2022-02-28 00:00:00...
executing for day 2022-03-31 00:00:00...
executing for day 2022-04-30 00:00:00...
executing for day 2022-05-31 00:00:00...
executing for day 2022-06-30 00:00:00...
executing for day 2022-07-31 00:00:00...
executing for day 2022-08-31 00:00:00...
executing for day 2022-09-30 00:00:00...
executing for day 2022-10-31 00:00:00...
executing for day 2022-11-30 00:00:00...
executing for day 2022-12-31 00:00:00...
executing for day 2023-01-31 00:00:00...
executing for day 2023-02-28 00:00:00...
executing for da

In [ ]:
USE IT OR LOOSE IT